In [1]:
import os
import pandas as pd
from price_loaders.tradingview import load_asset_price
import numpy as np

def save_to_csv(data, folder, filename):
    filepath = os.path.join(folder, filename)
    data.to_csv(filepath, index=False)
    print(f"Lưu dữ liệu thành công vào {filepath}")

In [2]:


# eu_gdp_growth                   = load_asset_price("EUGDPYY", 80, "3M", None) 
# eu_interest_rate                = load_asset_price("EUINTR", 240, "1M", None) 
# eu_inflation_rate               = load_asset_price("EUIRYY", 240, "1M", None)
# eu_consumer_price_index         = load_asset_price("EUCPI", 240, "1M", None)
# eu_producer_price_index         = load_asset_price("EUPPI", 240, "1M", None)
# eu_unemployment_rate            = load_asset_price("EUUR", 240, "1M", None)
# eu_trade_balance                = load_asset_price("EUBOT", 240, "1M", None)
# eu_gov_debt                     = load_asset_price("EUGDG", 10, "12M", None)
# eu_consumer_confidence_index    = load_asset_price("EUCCI", 240, "1M", None)
# eu_retail_sales                 = load_asset_price("EURSMM", 2,"1M", None)
# eu_money_supply                 = load_asset_price("EUM2", 2,"1M", None)



In [3]:
#EURUSD
eurusd_D = load_asset_price("EURUSD", 7560, "1D", None)  # ~30 năm ngày
eurusd_W = load_asset_price("EURUSD", 1560, "1W", None)  # ~30 năm tuần
eurusd_M = load_asset_price("EURUSD", 360, "1M", None)   # ~30 năm tháng

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def seasonality_monthly(df, start_year=2015, ignore_months=None):
    """
    Tính seasonal tendency theo tháng từng năm và vẽ heatmap cùng thống kê.

    Parameters:
    - df: DataFrame có 'time' (datetime), 'close' (float)
    - start_year: năm bắt đầu tính toán (bao gồm)
    - ignore_months: list các tháng dạng 'YYYY-MM' để loại bỏ (vd: ['2020-03', '2021-07'])

    Returns:
    - heatmap figure, stats DataFrame
    """
    # Xử lý datetime
    df['time'] = pd.to_datetime(df['time'])
    df = df[df['time'].dt.year >= start_year].copy()

    # Loại bỏ tháng bị ignore nếu có
    if ignore_months:
        ignore_months = set(ignore_months)
        df['year_month'] = df['time'].dt.strftime('%Y-%m')
        df = df[~df['year_month'].isin(ignore_months)]

    # Sắp xếp theo thời gian
    df = df.sort_values('time').reset_index(drop=True)

    # Tính % thay đổi tháng so với tháng trước
    df['pct_change'] = df['close'].pct_change() * 100

    # Tạo cột năm và tháng
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month

    # Tạo ma trận: index là năm, columns là tháng, giá trị là % change
    pivot = df.pivot(index='year', columns='month', values='pct_change')

    # Thống kê từng tháng theo năm: trung bình, độ lệch chuẩn, % positive
    monthly_avg = pivot.mean(axis=0)
    monthly_std = pivot.std(axis=0)
    monthly_pos_pct = (pivot > 0).sum(axis=0) / pivot.count(axis=0) * 100

    # Tạo heatmap
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    fig = go.Figure(data=go.Heatmap(
        z=pivot.values,
        x=month_names,
        y=pivot.index.astype(str),
        colorscale='RdYlGn',
        colorbar=dict(title='% Change'),
        hovertemplate='Year: %{y}<br>Month: %{x}<br>Change: %{z:.2f}%' 
    ))

    # Vẽ thêm các thống kê bên dưới heatmap dưới dạng biểu đồ cột
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_avg,
        name='Average % Change',
        marker_color='blue',
        yaxis='y2'
    ))
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_std,
        name='Std Dev',
        marker_color='orange',
        yaxis='y2'
    ))
    fig.add_trace(go.Bar(
        x=month_names,
        y=monthly_pos_pct,
        name='% Positive',
        marker_color='green',
        yaxis='y2'
    ))

    # Cấu hình trục y thứ 2 để vẽ thống kê
    fig.update_layout(
        title=f'Seasonal Monthly Changes since {start_year}',
        yaxis=dict(title='Yearly Monthly % Change', domain=[0.3, 1]),
        yaxis2=dict(title='Stats', domain=[0, 0.25], anchor='x'),
        barmode='group',
        height=700,
        legend=dict(orientation='h', yanchor='bottom', y=1.05, xanchor='right', x=1),
        margin=dict(t=50)
    )

    # Dữ liệu thống kê trả về
    stats_df = pd.DataFrame({
        'Month': month_names,
        'Average % Change': monthly_avg.values,
        'Std Dev': monthly_std.values,
        '% Positive': monthly_pos_pct.values
    })

    return fig, stats_df

# Ví dụ sử dụng:
import pandas as pd

# Giả sử eurusd_D là DataFrame của bạn, đã có cột time dạng datetime với timezone


fig, stats = seasonality_monthly(eurusd_M, start_year=2015)
fig.show()
print(stats)


   Month  Average % Change   Std Dev  % Positive
0    Jan          0.215407  1.763767   50.000000
1    Feb         -0.794183  0.967026   18.181818
2    Mar          0.320317  2.739209   54.545455
3    Apr          0.695844  2.836350   54.545455
4    May         -0.136085  2.281019   54.545455
5    Jun          0.453049  2.049188   54.545455
6    Jul          0.300458  2.340898   63.636364
7    Aug          0.078349  1.369035   40.000000
8    Sep         -0.918965  1.234643   30.000000
9    Oct         -0.762996  1.548052   30.000000
10   Nov         -0.060904  3.157265   50.000000
11   Dec          1.104395  1.571698   80.000000


In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def calculate_swing_high_low(df, timeframe='D'):
    """
    Tính swing high/low và lọc đỉnh/đáy hợp lệ theo logic đã cho.
    Trả về df với 2 cột mới: 'valid_swing_high', 'valid_swing_low' (bool).
    """
    df = df.reset_index(drop=True)

    if(timeframe == 'D'):
        window = 5
    elif(timeframe == 'W'):
        window = 4
    elif(timeframe == 'M'):
        window = 3
    else:
        raise ValueError("Invalid timeframe")
    highs = df['high']
    lows = df['low']

    swing_highs = []
    swing_lows = []
    n = len(df)

    # Tìm điểm swing high/low thô
    for i in range(n):
        start = max(i - window, 0)
        end = min(i + window + 1, n)

        max_around = highs[start:i].max() if i > start else -np.inf
        max_around = max(max_around, highs[i+1:end].max() if i+1 < end else -np.inf)

        min_around = lows[start:i].min() if i > start else np.inf
        min_around = min(min_around, lows[i+1:end].min() if i+1 < end else np.inf)

        swing_highs.append(highs[i] > max_around)
        swing_lows.append(lows[i] < min_around)

    df['swing_high'] = swing_highs
    df['swing_low'] = swing_lows

    swing_high_indices = df.index[df['swing_high']].tolist()
    swing_low_indices = df.index[df['swing_low']].tolist()

    # Hàm kiểm tra swing high hợp lệ
    def is_valid_swing_high(sh_idx):
        lows_before = [idx for idx in swing_low_indices if idx < sh_idx]
        lows_after = [idx for idx in swing_low_indices if idx > sh_idx]
        if not lows_before or not lows_after:
            return False
        low_before_idx = max(lows_before)
        low_after_idx = min(lows_after)
        high_price = df.at[sh_idx, 'high']
        low_before_price = df.at[low_before_idx, 'low']
        low_after_price = df.at[low_after_idx, 'low']
        return low_before_price < high_price and low_after_price < high_price

    # Hàm kiểm tra swing low hợp lệ
    def is_valid_swing_low(sl_idx):
        highs_before = [idx for idx in swing_high_indices if idx < sl_idx]
        highs_after = [idx for idx in swing_high_indices if idx > sl_idx]
        if not highs_before or not highs_after:
            return False
        high_before_idx = max(highs_before)
        high_after_idx = min(highs_after)
        low_price = df.at[sl_idx, 'low']
        high_before_price = df.at[high_before_idx, 'high']
        high_after_price = df.at[high_after_idx, 'high']
        return high_before_price > low_price and high_after_price > low_price

    valid_swing_highs = [(idx, is_valid_swing_high(idx)) for idx in swing_high_indices]
    valid_swing_lows = [(idx, is_valid_swing_low(idx)) for idx in swing_low_indices]

    df['valid_swing_high'] = False
    for idx, valid in valid_swing_highs:
        df.at[idx, 'valid_swing_high'] = valid

    df['valid_swing_low'] = False
    for idx, valid in valid_swing_lows:
        df.at[idx, 'valid_swing_low'] = valid

    # Lọc loại bỏ các đỉnh liên tiếp không hợp lệ
    def filter_consecutive_highs():
        valid_highs = df.index[df['valid_swing_high']].tolist()
        valid_lows = df.index[df['valid_swing_low']].tolist()
        filtered_highs = []

        i = 0
        while i < len(valid_highs):
            curr_high = valid_highs[i]

            if i == len(valid_highs) - 1:
                filtered_highs.append(curr_high)
                break

            next_high = valid_highs[i + 1]
            lows_between = [low for low in valid_lows if curr_high < low < next_high]

            if lows_between:
                filtered_highs.append(curr_high)
                i += 1
            else:
                curr_high_price = df.at[curr_high, 'high']
                next_high_price = df.at[next_high, 'high']

                if curr_high_price > next_high_price:
                    filtered_highs.append(curr_high)
                    i += 2  # bỏ next_high
                else:
                    i += 1  # bỏ curr_high, giữ next_high

        if valid_highs[-1] not in filtered_highs:
            filtered_highs.append(valid_highs[-1])

        df['valid_swing_high'] = False
        df.loc[filtered_highs, 'valid_swing_high'] = True

    # Lọc loại bỏ các đáy liên tiếp không hợp lệ
    def filter_consecutive_lows():
        valid_lows = df.index[df['valid_swing_low']].tolist()
        valid_highs = df.index[df['valid_swing_high']].tolist()
        filtered_lows = []

        i = 0
        while i < len(valid_lows):
            curr_low = valid_lows[i]

            if i == len(valid_lows) - 1:
                filtered_lows.append(curr_low)
                break

            next_low = valid_lows[i + 1]
            highs_between = [high for high in valid_highs if curr_low < high < next_low]

            if highs_between:
                filtered_lows.append(curr_low)
                i += 1
            else:
                curr_low_price = df.at[curr_low, 'low']
                next_low_price = df.at[next_low, 'low']

                if curr_low_price < next_low_price:
                    filtered_lows.append(curr_low)
                    i += 2  # bỏ next_low
                else:
                    i += 1  # bỏ curr_low, giữ next_low

        if valid_lows[-1] not in filtered_lows:
            filtered_lows.append(valid_lows[-1])

        df['valid_swing_low'] = False
        df.loc[filtered_lows, 'valid_swing_low'] = True

    filter_consecutive_highs()
    filter_consecutive_lows()

    return df


def calculate_moving_averages(df, windows=[30, 60, 90]):
    """
    Tính các đường MA với list windows (mặc định 30, 60, 90).
    Trả về df với các cột MA tương ứng.
    """
    for w in windows:
        df[f'MA{w}'] = df['close'].rolling(window=w).mean()
    return df


In [6]:
# Tính swing high/low hợp lệ
eurusd_D = calculate_swing_high_low(eurusd_D, timeframe='D')
# Tính các đường MA
eurusd_D = calculate_moving_averages(eurusd_D, windows=[30, 60, 90])
eurusd_D['time'] = pd.to_datetime(eurusd_D['time'])


In [7]:
# Tính swing high/low hợp lệ
eurusd_W = calculate_swing_high_low(eurusd_W, timeframe='W')
# Tính các đường MA
eurusd_W = calculate_moving_averages(eurusd_W, windows=[30, 60, 90])
eurusd_W['time'] = pd.to_datetime(eurusd_W['time'])

In [8]:
# Tính swing high/low hợp lệ
eurusd_M = calculate_swing_high_low(eurusd_M, timeframe='M')
# Tính các đường MA
eurusd_M = calculate_moving_averages(eurusd_M, windows=[30, 60, 90])
eurusd_M['time'] = pd.to_datetime(eurusd_M['time'])

In [9]:
max_date = eurusd_D['time'].max()

In [ ]:

def plot_candlestick(
    df,
    title="EURUSD Candlestick Chart",
    show_volume=False,
    show_swings=False,
    timeframe='D',  # D / W / M
    start_date=None,
    end_date=None
):
    """
    Vẽ biểu đồ nến cho EURUSD với các đường MA và tuỳ chọn volume + swing points.
    
    Args:
        df: DataFrame phải chứa các cột: time, open, high, low, close, MA30/60/90 (optional), volume (optional)
        title: Tiêu đề biểu đồ
        show_volume: Nếu True sẽ vẽ volume (dưới)
        show_swings: Nếu True sẽ vẽ các điểm swing high/low
        timeframe: 'D', 'W', 'M' (D: daily, weekly, monthly)
        start_date: chuỗi dạng 'YYYY-MM-DD' hoặc datetime để lọc ngày bắt đầu
        end_date: chuỗi dạng 'YYYY-MM-DD' hoặc datetime để lọc ngày kết thúc
    """

    # Lọc dữ liệu theo ngày nếu có
    if start_date:
        start_date = pd.to_datetime(start_date).tz_localize(df['time'].dt.tz)
    if end_date:
        end_date = pd.to_datetime(end_date).tz_localize(df['time'].dt.tz)

    # Lọc theo ngày nếu có
    if start_date or end_date:
        df = df.copy()
        if start_date:
            df = df[df['time'] >= start_date]
        if end_date:
            df = df[df['time'] <= end_date]

    fig = go.Figure()

    # Candlestick
    fig.add_trace(go.Candlestick(
        x=df['time'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        increasing_line_color='green',
        decreasing_line_color='black',
        name=f'{timeframe} Candlestick'
    ))

    # MA lines
    for ma, color in zip(['MA30', 'MA60', 'MA90'], ['blue', 'orange', 'green']):
        if ma in df.columns:
            fig.add_trace(go.Scatter(
                x=df['time'],
                y=df[ma],
                mode='lines',
                line=dict(color=color, width=1.5),
                name=ma
            ))

    # Swing High / Low
    if show_swings:
        if 'valid_swing_high' in df.columns:
            valid_sh = df[df['valid_swing_high']]
            fig.add_trace(go.Scatter(
                x=valid_sh['time'],
                y=valid_sh['high'],
                mode='markers',
                marker=dict(color='purple', size=10, symbol='star-triangle-up'),
                name='Swing High'
            ))
        if 'valid_swing_low' in df.columns:
            valid_sl = df[df['valid_swing_low']]
            fig.add_trace(go.Scatter(
                x=valid_sl['time'],
                y=valid_sl['low'],
                mode='markers',
                marker=dict(color='brown', size=10, symbol='star-triangle-down'),
                name='Swing Low'
            ))

    # Volume
    if show_volume and 'volume' in df.columns:
        volume_colors = ['green' if c >= o else 'black' for o, c in zip(df['open'], df['close'])]
        fig.add_trace(go.Bar(
            x=df['time'],
            y=df['volume'],
            name='Volume',
            marker_color=volume_colors,
            yaxis='y2',
            opacity=0.6
        ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            domain=[0, 1],
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=2, label="2Y", step="year", stepmode="backward"),
                    dict(count=5, label="5Y", step="year", stepmode="backward"),
                    dict(count=10, label="10Y", step="year", stepmode="backward"),
                    dict(count=20, label="20Y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(visible=True),
            type="date"
        ),
        yaxis=dict(
            title='Price',
            domain=[0.3, 1] if show_volume else [0, 1],
            autorange=True
        ),
        yaxis2=dict(
            title='Volume',
            domain=[0, 0.25],
            autorange=True,
            showgrid=False
        ) if show_volume else None,
        xaxis_rangeslider_visible=False,
        template='plotly_white',
        height=500,
        width=1000,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig.show()



def plot_seasonal_tendency(df):
    df = df.copy()
    df['time'] = pd.to_datetime(df['time'])
    max_year = df['time'].dt.year.max()
    
    # Tính pct_change trên toàn bộ df, theo thứ tự thời gian
    df = df.sort_values('time').reset_index(drop=True)
    df['pct_change'] = df['close'].pct_change() * 100

    periods = [
        (max_year - 20, 'Last 20 Years'),
        (max_year - 10, 'Last 10 Years'),
        (max_year - 5,  'Last 5 Years'),
        (max_year - 2,  'Last 2 Years')
    ]

    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    data = []

    for start_year, label in periods:
        temp_df = df[df['time'].dt.year >= start_year].copy()
        temp_df['year'] = temp_df['time'].dt.year
        temp_df['month'] = temp_df['time'].dt.month

        pivot = temp_df.pivot(index='year', columns='month', values='pct_change')

        monthly_avg = pivot.mean(axis=0)

        for month_idx, avg_val in monthly_avg.items():
            data.append({
                'Month': month_names[month_idx - 1],
                'Average % Change': avg_val,
                'Period': label
            })

    df_plot = pd.DataFrame(data)

    fig = px.line(df_plot, x='Month', y='Average % Change', color='Period',
                  title='Seasonal Tendency by Month',
                  markers=True,
                  labels={'Average % Change': '% Change'})

    fig.update_layout(
        yaxis_tickformat='.2f',
        template='plotly_white',
        height=500,
        width=1000,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig.show()


In [12]:
import cot_reports as cot
legacy_fut, legacy_futopt, supplemental_futopt, disaggregated_fut, disaggregated_futopt, traders_fin_fut_fut, traders_fin_fut_futopt = cot.cot_all_reports()

eur_fx_futures_cme = legacy_fut[
    legacy_fut['Market and Exchange Names'].str.contains("EURO FX - CHICAGO MERCANTILE EXCHANGE", case=False)
]

legacy_fut
Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2017
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2018
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2019
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2020
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2021
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in 

In [40]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def plot_cot_index_with_extremes_plotly(df, start_date='2025-01-01', end_date=None, weeks=26, 
                                        upperExtreme=80, lowerExtreme=20):
    df = df.copy()
    df.rename(columns={
        'As of Date in Form YYYY-MM-DD': 'date',
        'Commercial Positions-Long (All)': 'commercial_long',
        'Commercial Positions-Short (All)': 'commercial_short',
        'Noncommercial Positions-Long (All)': 'noncommercial_long',
        'Noncommercial Positions-Short (All)': 'noncommercial_short',
        'Nonreportable Positions-Long (All)': 'retail_long',
        'Nonreportable Positions-Short (All)': 'retail_short',
        'Open Interest (All)': 'open_interest',
    }, inplace=True)

    df['date'] = pd.to_datetime(df['date'])

    # 1. Resample dữ liệu theo tuần (thứ Sáu)
    df_weekly = df.set_index('date').resample('W-FRI').last().dropna().reset_index()

    # 2. Tính net positions
    df_weekly['net_commercial'] = df_weekly['commercial_long'] - df_weekly['commercial_short']
    df_weekly['net_large'] = df_weekly['noncommercial_long'] - df_weekly['noncommercial_short']
    df_weekly['net_retail'] = df_weekly['retail_long'] - df_weekly['retail_short']

    # 3. Normalize theo open interest
    df_weekly['adj_commercial'] = df_weekly['net_commercial'] / df_weekly['open_interest'].replace(0, np.nan)
    df_weekly['adj_large'] = df_weekly['net_large'] / df_weekly['open_interest'].replace(0, np.nan)
    df_weekly['adj_retail'] = df_weekly['net_retail'] / df_weekly['open_interest'].replace(0, np.nan)

    # 4. Tính chỉ số COT Index
    def calc_index(adj_net_pos):
        min_val = adj_net_pos.rolling(window=weeks, min_periods=1).min()
        max_val = adj_net_pos.rolling(window=weeks, min_periods=1).max()
        idx = 100 * (adj_net_pos - min_val) / (max_val - min_val)
        idx[max_val == min_val] = np.nan
        return idx

    df_weekly['cot_index_commercial'] = calc_index(df_weekly['adj_commercial'])
    df_weekly['cot_index_large'] = calc_index(df_weekly['adj_large'])
    df_weekly['cot_index_retail'] = calc_index(df_weekly['adj_retail'])

    # 5. Lọc theo thời gian
    if end_date:
        mask = (df_weekly['date'] >= pd.to_datetime(start_date)) & (df_weekly['date'] <= pd.to_datetime(end_date))
    else:
        mask = df_weekly['date'] >= pd.to_datetime(start_date)
    df_ = df_weekly.loc[mask].copy()

    # 6. Vùng cực trị
    def find_regions(mask, dates):
        regions = []
        start = None
        for i, val in enumerate(mask):
            if val and start is None:
                start = dates.iat[i]
            elif not val and start is not None:
                end = dates.iat[i-1]
                regions.append((start, end))
                start = None
        if start is not None:
            regions.append((start, dates.iat[-1]))
        return regions

    bullish_mask = (df_['cot_index_commercial'] >= upperExtreme) & (df_['cot_index_retail'] <= lowerExtreme)
    bearish_mask = (df_['cot_index_commercial'] <= lowerExtreme) & (df_['cot_index_retail'] >= upperExtreme)

    bullish_regions = find_regions(bullish_mask.values, df_['date'])
    bearish_regions = find_regions(bearish_mask.values, df_['date'])

    # 7. Plot bằng Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_commercial'],
                             mode='lines', name='Commercial', line=dict(color='green')))
    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_large'],
                             mode='lines', name='Large Speculators', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df_['date'], y=df_['cot_index_retail'],
                             mode='lines', name='Retail', line=dict(color='blue')))

    # Vùng cực trị
    for start, end in bullish_regions:
        fig.add_vrect(x0=start, x1=end, fillcolor='blue', opacity=0.2, line_width=0)
    for start, end in bearish_regions:
        fig.add_vrect(x0=start, x1=end, fillcolor='red', opacity=0.2, line_width=0)

    # Đường ngưỡng
    fig.add_hline(y=upperExtreme, line_dash="dash", line_color="gray")
    fig.add_hline(y=lowerExtreme, line_dash="dash", line_color="gray")

    fig.update_layout(
        title=f"CoT Index with Extreme Zones ({start_date} to {end_date or 'latest'})",
        xaxis_title="Date",
        yaxis_title="CoT Index (%)",
        template="plotly_white",
        height=500,
        width=1000,
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()


In [ ]:
df = eur_fx_futures_cme[eur_fx_futures_cme['Market and Exchange Names'] == "EURO FX - CHICAGO MERCANTILE EXCHANGE"].copy()

In [ ]:
plot_candlestick(eurusd_M, title="EURUSD - Monthly Chart", show_volume=True, show_swings=True, timeframe='M', start_date='2025-01-01', end_date='2025-07-21')
plot_candlestick(eurusd_W, title="EURUSD - Weekly Chart", show_volume=True, show_swings=True, timeframe='W', start_date='2025-01-01', end_date='2025-07-21')
plot_candlestick(eurusd_D, title="EURUSD - Daily Chart", show_volume=True, show_swings=True, timeframe='D', start_date='2025-01-01', end_date='2025-07-21')
plot_cot_index_with_extremes_plotly(df, weeks=26, upperExtreme=80, lowerExtreme=20, start_date='2025-01-01', end_date='2025-07-21', )
plot_seasonal_tendency(eurusd_M)
